In [6]:
from sklearn.grid_search import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit
from sklearn import linear_model
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import log_loss, mean_squared_error as mse, r2_score, make_scorer 
from sklearn.metrics.scorer import make_scorer

In [7]:
def CVScore(model, X_train, y_train, n_splits=5, is_TimeSeries=False, seed=2017, my_score=mse):
    cv_scores = []
    X_train=np.array(X_train)
    y_train=np.array(y_train)
    if not is_TimeSeries:
        kf=KFold(n_splits=n_splits, shuffle=True, random_state=17)
    else:
        kf=TimeSeriesSplit(n_splits=n_splits)
    for train_idx, test_idx in kf.split(X_train):
        X_CVtrain = X_train[train_idx]
        y_CVtrain = y_train[train_idx]
        X_CVholdout = X_train[test_idx]
        y_CVholdout = y_train[test_idx]
        model.fit(X_CVtrain, y_CVtrain)
        if my_score==log_loss:
            pred=model.predict_proba(X_CVholdout)
        else:
            pred = model.predict(X_CVholdout)[:]
        cv_scores.append(my_score(y_CVholdout, pred))
    return np.mean(cv_scores)

In [8]:
n=1000
np.random.seed(17)
X=pd.DataFrame(np.random.randn(n,1))
y=X.iloc[:,0]+.2*pd.Series(np.random.randn(n))
X_train,y_train=X.iloc[:n/2], y.iloc[:n/2]
X_test, y_test=X.iloc[n/2:], y.iloc[n/2:]

In [9]:
CVScore(linear_model.LinearRegression(fit_intercept=False), X_train, y_train, my_score=mse)

0.043842510288573847

In [ ]:
def RMSLE_(y_val, y_val_pred):
    return np.sqrt(np.mean((np.log(y_val+1)-np.log(y_val_pred+1))**2))
RMSLE = make_scorer(RMSLE_, greater_is_better=False) 

In [ ]:
xgbreg = xgb.XGBRegressor()
param_grid = {
       #'n_estimators': [500],
       'learning_rate': [ 0.05],
       'max_depth': [5],
       'subsample': [ 0.7],
       'colsample_bytree': [0.7],
        'seed':[123,2017,2]
}
model = GridSearchCV(estimator=xgbreg, param_grid=param_grid, n_jobs=-1, cv=3, scoring=RMSLE)
model.fit(X_train, y_train)
print('eXtreme Gradient Boosting regression...')
print('Best Params:')
print(model.best_params_)
print('Best CV Score:')
print(-model.best_score_)